[View in Colaboratory](https://colab.research.google.com/github/gopal2812/mlblr/blob/master/seconditerationdensenetcifar10.ipynb)

In [1]:
!pip install -q keras
import keras

import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

batch_size = 128
num_classes = 10
epochs = 50
l = 24
num_filter = 64
compression = 0.5
dropout_rate = 0.5

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp
  
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg
  
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output
  
num_filter = 64
dropout_rate = 0.2
l = 6
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

model = Model(inputs=[input], outputs=[output])
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.


170500096/170498071 [==============================] - 16s 0us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 64)   1728        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 64)   0           batch_normalization_1[0][0]      
_________________________________________

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 197s 4ms/step - loss: 1.3764 - acc: 0.4988 - val_loss: 2.1251 - val_acc: 0.4105
Epoch 2/50
19200/50000 [==========>...................] - ETA: 1:47 - loss: 0.9890 - acc: 0.6460

50000/50000 [==============================] - 187s 4ms/step - loss: 0.9310 - acc: 0.6676 - val_loss: 1.3788 - val_acc: 0.5932
Epoch 3/50
45568/50000 [==========================>...] - ETA: 15s - loss: 0.7491 - acc: 0.7337

50000/50000 [==============================] - 188s 4ms/step - loss: 0.7476 - acc: 0.7342 - val_loss: 0.8431 - val_acc: 0.7152
Epoch 4/50
50000/50000 [==============================] - 187s 4ms/step - loss: 0.6526 - acc: 0.7695 - val_loss: 1.4238 - val_acc: 0.5866
Epoch 5/50
 3328/50000 [>.............................] - ETA: 2:42 - loss: 0.6331 - acc: 0.7755

...

50000/50000 [==============================] - 186s 4ms/step - loss: 0.1008 - acc: 0.9641 - val_loss: 0.6647 - val_acc: 0.8570
Epoch 45/50
38656/50000 [======================>.......] - ETA: 39s - loss: 0.0983 - acc: 0.9648

50000/50000 [==============================] - 186s 4ms/step - loss: 0.0989 - acc: 0.9645 - val_loss: 0.7396 - val_acc: 0.8465
Epoch 46/50
50000/50000 [==============================] - 186s 4ms/step - loss: 0.0991 - acc: 0.9646 - val_loss: 0.6375 - val_acc: 0.8613
Epoch 47/50
 1536/50000 [..............................] - ETA: 2:47 - loss: 0.0906 - acc: 0.9681

50000/50000 [==============================] - 186s 4ms/step - loss: 0.0909 - acc: 0.9672 - val_loss: 0.5755 - val_acc: 0.8756
Epoch 48/50
38656/50000 [======================>.......] - ETA: 39s - loss: 0.0892 - acc: 0.9680

50000/50000 [==============================] - 186s 4ms/step - loss: 0.0940 - acc: 0.9661 - val_loss: 0.6726 - val_acc: 0.8561
Epoch 49/50
50000/50000 [==============================] - 186s 4ms/step - loss: 0.0882 - acc: 0.9690 - val_loss: 0.6432 - val_acc: 0.8692
Epoch 50/50
 1536/50000 [..............................] - ETA: 2:47 - loss: 0.0758 - acc: 0.9707

10000/10000 [==============================] - 17s 2ms/step
Test loss: 0.5964930905386806
Test accuracy: 0.8735
